# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Загрузка библиотек

In [ ]:
!pip install -U scikit-learn -q

### Импотр библиотек

In [ ]:
import os
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

try:
    import spacy
except:
    !pip install spacy -q
    import spacy

from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

### Загрузка данных

In [ ]:
df = pd.read_csv('/datasets/toxic_comments.csv', usecols=[1, 2])

display(df.head(10))
print('----------\nИнформация о таблице:\n')
df.info()
print(f'----------\nДубликатов в таблице: {df.duplicated().sum()}')

### Токенизация, удаление стоп слов, лематизация, удаление строк разделителей

In [ ]:
%%time

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stopwords_english = set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')

def f_preparation(text):
    # в нижний регистр
    text = text.lower()
    # только текст
    text = re.sub(r'[^a-z]', ' ', text)
    # удаление повторных пробелов
    text = re.sub(r'\s+', ' ', text)
    # деление текста на токены
    text = nltk.word_tokenize(text)
    text = ' '.join(text)
    return text

df['lemmatize_text'] = df['text'].apply(f_preparation)
df.head(10)

In [ ]:
%%time

wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

def f_lemmatize_sent(text): 
    # Вводимый текст представляет собой строку, возвращающую строки в нижнем регистре.
    text = [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) for word, tag in pos_tag(nltk.word_tokenize(text))]
    # удаление стоп слов
    text = [word for word in text if word not in stopwords_english]
    return ' '.join(text)
     
df['lemmatize_text'] = df['lemmatize_text'].apply(f_lemmatize_sent)
df.head(10)

In [ ]:
def f_conti_rep_char(str1):
    tchr = str1.group(0)
    if len(tchr) > 1:
      return tchr[0:1]

def f_check_unique_char(rep, sent_text):
    # регулярное выражение для повторяющихся символов
    convert = re.sub(r'(\w)\1+', rep, sent_text)
    # возврат конвертированного слова
    return convert
 
df['lemmatize_text'] = df['lemmatize_text'].apply(lambda x : f_check_unique_char(f_conti_rep_char, x))
df

### Проверка разделения по классам

In [ ]:
df['toxic'].value_counts().plot.bar()
plt.title('Соотношение токсичности комментариев')
plt.xlabel('Доля комментраиев "1 = токсичный"', fontsize=10, color='blue')
plt.ylabel("Количество коментариев", fontsize=10, color='orange')
plt.show();

**Выводы по этапу предварительной обработки информации**

В ходе реализации подготовительного этапа были выполнены следующие операции:
- Интеграция программных инструментов: осуществлён процесс подключения необходимых библиотек и модулей
- Загрузка информационных массивов: выполнен импорт исходных данных для последующей обработки
- Комплексная очистка данных: реализован многоступенчатый процесс подготовки информации, включающий:
    - Нормализация текста: приведение всех текстовых элементов к единому регистру
    - Фильтрация контента: удаление неалфавитных символов и специальных знаков
    - Морфологический анализ: разбивка текстовых последовательностей на составные элементы
    - Лемматизация: приведение слов к исходной форме
    - Семантическая очистка: исключение служебных слов и стоп-символов
- Статистический анализ: проведено исследование распределения показателей токсичности по категориям

По завершении подготовительного этапа формируется база для перехода к следующему этапу работы — построению и обучению аналитических моделей.

## Обучение

### Разделение на обучающую и тестовую выборки

Для отбора лучшей модели сократим выборку

In [ ]:
data = df.sample(n=10000, random_state=42).copy()

In [ ]:
RANDOM_STATE = 42
# инициализация TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

X = data['lemmatize_text']
y = data['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify = y)
round(X_train.shape[0]/X.shape[0], 2), round(X_test.shape[0]/X.shape[0], 2)

### Создадим общий `Pipeline`

In [ ]:
def f_training(model, params):
    pipeline = Pipeline([
        ('vect', TfidfVectorizer()),
        ('model', model)])
    grid = GridSearchCV(pipeline, cv = 5, n_jobs = -1, param_grid = params ,scoring = 'f1', verbose = False)
    grid.fit(X_train, y_train)
    print(f'Значение "F1-score" на кросс-валидации: {grid.best_score_:.3}\n----------')
    print(f'Параметры лучшей модели: {grid.best_params_}\n----------')
    return grid

### Обучим `LogisticRegression()`

In [ ]:
%%time
lr_mod = f_training(LogisticRegression(random_state=RANDOM_STATE), 
                    {
                     'model__C':[0.1, 1.0, 10.0], 
                     'model__penalty':["l1", "l2", "elasticnet", None]
                    }
)

### Обучим `SVC`

In [ ]:
%%time
svc_mod = f_training(SVC(kernel='linear', 
                     random_state=RANDOM_STATE),
                     {
                      'model__degree':[3, 4]
                     }
)

### Обучим `DecisionTreeClassifier()`

In [ ]:
%%time
dtc_mod = f_training(DecisionTreeClassifier(criterion='gini', random_state=RANDOM_STATE),
                     {
                      'model__max_depth':[None, 2,4,8]
                     }
)

### Обучим `LGBMClassifier()`

In [ ]:
%%time
lgbm_mod = f_training(LGBMClassifier(learning_rate=0.1, 
                                     n_estimators=200, 
                                     random_state=RANDOM_STATE,
                                     n_jobs=1,
                                     verbose=-1), 
                                     {
                                      'model__max_depth': [None, 8]
                                     }
)

### Составим сводную таблицу по обученным моделям

Отберем модель с максимальным значением `F1-score` на валидационной выборке

In [ ]:
results = pd.DataFrame({'Model': ['LogisticRegression', 'SVC', 'DecisionTreeClassifier', 'LGBMClassifier'],
                        'f1_score': [lr_mod.best_score_, svc_mod.best_score_, dtc_mod.best_score_, lgbm_mod.best_score_]
                      })
results = results.sort_values('f1_score', ascending=False).reset_index(drop=True).head(1)
results

### Переобучим выбранную модель на полных данных

Подготовка данных

In [ ]:
X = df['lemmatize_text']
y = df['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify = y)
round(X_train.shape[0]/X.shape[0], 2), round(X_test.shape[0]/X.shape[0], 2)

In [ ]:
%%time
lr_mod = f_training(LogisticRegression(random_state=RANDOM_STATE), 
                    {
                     'model__C':[0.1, 1.0, 10.0], 
                     'model__penalty':["l1", "l2", "elasticnet", None]
                    }
)

### Расчитаем `F1-score` на тестовой выборке

В соответствии с условием задачи её значение должно быть не меньше 0,75

In [ ]:
print(f'Метрика F1-score "LogisticRegression" на тестовой выборке: {lr_mod.score(X_test, y_test):.3f}')

### Проверим лучшую модель на адекватность моделью `DummyClassifier`

In [ ]:
dummy_clf = DummyClassifier(strategy='stratified', random_state=RANDOM_STATE).fit(X_train, y_train)

print(f'Метрика F1-score "DummyClassifier" на тестовой выборке: {f1_score(y_test, dummy_clf.predict(X_test)):.3f}')
print(f'Значение F1-score "LogisticRegression" на тестовой выборке лучше "DummyClassifier" в: {lr_mod.score(X_test, y_test) / f1_score(y_test, dummy_clf.predict(X_test)) :.3f} раз')

**Вывод**

Полученные показатели валидации разработанного алгоритма на проверочной подвыборке демонстрируют превосходство над базовым решением, построенным на константных значениях. Таким образом, осуществлённая исследовательская работа и реализованные технические решения позволили достичь поставленных целей проекта и получить значимый практический результат.

Успешное превышение метрик по сравнению с тривиальной моделью подтверждает эффективность выбранных методологических подходов и корректность проведённой оптимизации параметров системы.

## Выводы

В ходе реализации исследовательского проекта были последовательно выполнены следующие этапы:

- **Предварительная обработка массива данных**, включающая:
    - Унификацию регистра текстовых элементов
    - Фильтрацию символьного состава с сохранением исключительно буквенных значений
    - Выполнение процедуры токенизации
    - Приведение лексем к базовой форме (лемматизация)
    - Элиминацию стоп-слов
    - Исследование распределения показателей токсичности по категориальным признакам

- **Стратификация датасета**: осуществлено разделение информационного массива на подмножества для обучения и валидации. С целью оптимизации процесса поиска оптимальной конфигурации произведено случайное сокращение объёма выборки до 10 000 записей.

- **Имплементация алгоритмов классификации**:
    - Логистическая регрессия (`LogisticRegression`)
    - Метод опорных векторов (`SVC`)
    - Деревья решений (`DecisionTreeClassifier`)
    - Градиентный бустинг (`LGBMClassifier`)

- **Оценка эффективности**: на основе метрики F1-score (комплексный показатель качества для задач классификации, рассчитываемый как гармоническое среднее точности и полноты) была идентифицирована оптимальная модель — `LogisticRegression`.

- **Финальная валидация**: после переобучения выбранной модели на полном обучающем наборе получены следующие результаты:
    - Значение F1-score для `LogisticRegression` при кросс-валидации: 0.76 (превышает целевой показатель 0.75)
    - Значение F1-score для `LogisticRegression` на валидационной выборке: 0.76 (превышает целевой показатель 0.75)

- **Верификация результатов**: проведено сопоставление расчётных метрик с целевым значением F1-score (не менее 0.75)

- **Проверка адекватности**: выполнена валидация оптимальной модели посредством сравнения с базовой моделью DummyClassifier

**Заключение**

Для интернет-магазин «Викишоп» разработан инструмент, который будет искать токсичные комментарии и отправлять их на модерацию, при этом значение качества модели превышает установленный минимальный уровень в 0.75

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны